In [1]:
import numpy as np
import pandas as pd

In [2]:
data_simple = pd.read_csv('data_simple.csv', encoding = "ISO-8859-1", low_memory=False, index_col=0)

In [8]:
def getDfSummary(input_data):
    summary = input_data.describe().transpose()
    summary = summary.drop(['count'], axis=1)

    number_distinct = input_data.unstack().groupby(level=0).nunique(dropna=True)
    number_distinct.name = "number_distinct"

    number_nan = input_data.isnull().sum()
    number_nan.name = "number_nan"
    
    data_type = input_data.dtypes
    data_type.name = "type"

    output_data = pd.concat([data_type,summary, number_distinct, number_nan], axis=1)

    
    return output_data

getDfSummary(data_simple)

,type,mean,std,min,25%,50%,75%,max,number_distinct,number_nan
AREG_SEQ_NBR,int64,1.000000e+00,0.000000e+00,1.0,1.00,1.0,1.00,1.0,1,0
ARREST_CREDIT_CODE,object,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,133502
ARREST_DATE,float64,1.955671e+07,2.748079e+06,0.0,19910830.00,19941025.0,19970731.00,19991115.0,4259,129315
BOFI_NBR,object,NaN,NaN,NaN,NaN,NaN,NaN,NaN,124848,1
CHARGE_CAT,object,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,229739
CHARGE_CLASS,float64,3.328816e+00,8.820815e-01,1.0,3.00,3.0,4.00,8.0,8,131749
CHARGE_TYPE,object,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,129846
CRIMINAL_FLAG,object,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0
DFDN_SEQ_NBR,int64,1.097765e+00,4.838954e-01,1.0,1.00,1.0,1.00,27.0,27,0
DOB,int64,1.934283e+07,2.580123e+06,0.0,19610130.00,19701029.0,19770817.00,19990901.0,19779,0


In [13]:
#verify that 'BOFI_NBR','DFDN_SEQ_NBR','SYS_NBR', 'SADA_CODE', 'AREG_SEQ_NBR' uniquely identify this table
#each row represents an arrest
data_simple[data_simple.duplicated(['BOFI_NBR','DFDN_SEQ_NBR','SYS_NBR', 'SADA_CODE', 'AREG_SEQ_NBR'],keep=False)]

,SADA_CODE,AREG_SEQ_NBR,BOFI_NBR,DFDN_SEQ_NBR,SCREENING_DISP_CODE,SYS_NBR,POLICE_RPT_DATE,POLICE_RPT_DAYS,SCREENING_DAYS,SCREENING_DISP_DATE,...,SADA_PARTY,SADA_ALT_PARTY,NAME,RESTRICTED_FLAG,JUVENILE_FLAG,CRIMINAL_FLAG,FBI_NBR,DOB,SEX,RACE


In [17]:
data_simple['CHRG_ACCEPTED'] = np.where(data_simple['SCREENING_DISP_CODE']==230,1,0)
data_simple['CHRG_REJECTED'] = np.where(data_simple['SCREENING_DISP_CODE']!=230,1,0)

In [37]:
by_sada = data_simple.groupby(['SADA_CODE'])['CHRG_ACCEPTED','CHRG_REJECTED'].sum()

In [38]:
by_sada['CHRG_RATE'] = by_sada['CHRG_ACCEPTED']/(by_sada['CHRG_ACCEPTED'] + by_sada['CHRG_REJECTED'] )

In [39]:
by_sada

,CHRG_ACCEPTED,CHRG_REJECTED,CHRG_RATE
SADA_CODE,,,
ABAL,1,0,1.000000
ADAI,190,771,0.197711
ADIC,179,490,0.267564
AELS,6,0,1.000000
AGOU,2,0,1.000000
AGRN,5,0,1.000000
AGUS,2,0,1.000000
AJEA,16,0,1.000000
ALAM,0,67,0.000000
